In [3]:
from llama_cpp import Llama

llm = Llama(
    model_path="../models/mistral-7b.Q4_K_M.gguf",
    n_ctx=4096,
    n_threads=4,  # Replace with your actual number from `nproc`
    verbose=True
)


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ../models/mistral-7b.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attentio

In [4]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.document_loaders import PyPDFLoader

# Load document (PDF)
loader = PyPDFLoader("../data/A-man-called-Ove.pdf")
documents = loader.load()

# Embed and store
embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(documents, embedding, persist_directory="./chroma_db")
# vectordb.persist()


/tmp/ipykernel_4858/580222309.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [5]:
from langchain.chains import RetrievalQA
from langchain_community.llms import LlamaCpp  # Updated import
from langchain_community.vectorstores import Chroma  # Ensure you're using the correct Chroma class

# Load your GGUF model with llama-cpp
llm = LlamaCpp(
    model_path="../models/mistral-7b.Q4_K_M.gguf",
    n_ctx=4096,
    n_threads=4  # or 8 if your CPU supports it
)

# Use the retriever from your existing vector DB
retriever = vectordb.as_retriever()

# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # This is often required in latest LangChain
    retriever=retriever,
    return_source_documents=True
)

# Ask your question
response = qa_chain.invoke("What is this document about?")
print(response["result"])


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ../models/mistral-7b.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attentio

 This document appears to be a book cover for the novel "A Man Called Ove" by Fredrik Backman, translated from Swedish into English by Henning Koch. The title, author's name, and ISBN numbers are provided, along with some copyright information and details about the imprint that published the book. The cover features an image of a man looking out the window and appears to be set in a residential area.


In [6]:
response = qa_chain.invoke("what does he gift his neighbour?")
print(response["result"])

Llama.generate: 45 prefix-match hit, remaining 2050 prompt tokens to eval
llama_perf_context_print:        load time =  158351.79 ms
llama_perf_context_print: prompt eval time =  292481.07 ms /  2050 tokens (  142.67 ms per token,     7.01 tokens per second)
llama_perf_context_print:        eval time =    2578.30 ms /    13 runs   (  198.33 ms per token,     5.04 tokens per second)
llama_perf_context_print:       total time =  295066.13 ms /  2063 tokens


 He doesn't gift his neighbor anything in the text provided.


In [7]:
response = qa_chain.invoke("does he die in the end?")
print(response["result"])

Llama.generate: 45 prefix-match hit, remaining 1906 prompt tokens to eval
llama_perf_context_print:        load time =  158351.79 ms
llama_perf_context_print: prompt eval time =  344541.64 ms /  1906 tokens (  180.77 ms per token,     5.53 tokens per second)
llama_perf_context_print:        eval time =   14510.92 ms /    50 runs   (  290.22 ms per token,     3.45 tokens per second)
llama_perf_context_print:       total time =  359082.02 ms /  1956 tokens


 No, he does not die in the end. He is a grumpy old man who has recently lost his wife and is struggling to adjust to life without her. He becomes frustrated with his new neighbors and their behavior, but he does not die.
